In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Charan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("wayfair_uniq.csv")
df_new=df.iloc[:,[3,4,5,7,9,11,13,15,17,18,19]]

In [3]:
df_new['index'] = df_new.index

In [4]:
df_new.shape

(17209, 12)

In [5]:
df_new.head()

,product_name,product_name_link,manufacturer,price,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,from $38.02; $38.02,$69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,NaN,0
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,$73.99; $73.99,$99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,from $104.99; $104.99,$399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,$143.99; $143.99,$230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,$62.99; $62.99,$170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4


In [6]:
def get_title_from_index(index):
    return df_new[df_new.index == index]["product_name"].values[0]

def get_home(index):
    return df_new[df_new.index == index]["product_name_link"].values[0]

def get_index_from_title(title):
    return df_new[df_new.product_name == title]["index"].values[0]

In [7]:
print (df_new.columns)

Index(['product_name', 'product_name_link', 'manufacturer', 'price',
       'original_price', 'review_count', 'review_data', 'shipping_time',
       'product_descrip', 'product_data1', 'product_data2', 'index'],
      dtype='object')


In [8]:
df_new['original_price'] = df_new.original_price.str.replace('$', '')
df_new['original_price'] = df_new.original_price.str.replace(',', '').astype(float)

df_new['original_price'] 

0         69.99
1         99.99
2        399.99
3        230.00
4        170.04
          ...  
17204    205.99
17205    649.00
17206       NaN
17207    279.99
17208    569.99
Name: original_price, Length: 17209, dtype: float64

In [9]:
new_price = df_new["price"].str.split(";\s\$", n = 1, expand = True)
df_new["new_price"] = new_price[1]
df_new.drop(columns =["price"], inplace = True)
df_new["new_price"]

0         38.02
1         73.99
2        104.99
3        143.99
4         62.99
          ...  
17204    189.99
17205    189.99
17206    306.99
17207    265.99
17208    499.99
Name: new_price, Length: 17209, dtype: object

In [10]:
price_range = df_new[df_new["new_price"].str.contains("-",regex=True)]
price_range

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price
5898,Elissa Lounge Settee,https://www.wayfair.com/furniture/pdp/willa-ar...,by Willa Arlo Interiors,679.00,550.0,Rated 4.5 out of 5 stars.550 total votes.,FREE Shipping,Looking for a couch that adds a touch of Frenc...,Overall Product Weight: 83 lb.,Assembly Required: Yes,5898,479.99 - $719.99
6859,Jackson Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,2299.99,23.0,Rated 4.5 out of 5 stars.23 total votes.,FREE Shipping,The Jackson Sofa from the Jackson Collection c...,Design: Standard,NaN,6859,"1,949.00 - $3,637.00"
6879,Cheyenne Leather Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,2759.99,5.0,Rated 5 out of 5 stars.5 total votes.,FREE Shipping,Cheyenne Leather Sofa from the Cheyenne Collec...,NaN,NaN,6879,"2,569.99 - $4,249.99"
6988,Sedona Sleeper Sofa,https://www.wayfair.com/commercial/pdp/omnia-l...,by Omnia Leather,3059.99,7.0,Rated 5 out of 5 stars.7 total votes.,FREE Shipping,NaN,Overall: 96'' W x 39'' D,Primary Material Details: 100% top grain leather,6988,"2,899.99 - $4,539.99"
7027,Savannah Sleeper Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,NaN,4.0,Rated 4.5 out of 5 stars.4 total votes.,FREE Shipping,The Savannah Leather Sofa Bed 3 seat sofa will...,NaN,NaN,7027,"2,239.99 - $4,169.99"
7272,Torre Leather Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,NaN,2.0,Rated 4.5 out of 5 stars.2 total votes.,FREE Shipping,The antique finishing of the Torre Collection ...,Design: Sofa Bed; Standard,NaN,7272,"2,669.99 - $4,259.99"
7363,Nevada Leather Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,NaN,8.0,Rated 5 out of 5 stars.8 total votes.,FREE Shipping,The beauty of the Nevada collection by Omnia L...,Design: Standard,Overall: 88'' W x 37'' D,7363,"2,250.00 - $3,230.00"
7735,Oregon Leather Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,2659.99,5.0,Rated 5 out of 5 stars.5 total votes.,FREE Shipping,Oregon Sofa brings casual comfort into your ho...,NaN,NaN,7735,"2,450.00 - $3,100.00"
7940,Prescott Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,NaN,8.0,Rated 5 out of 5 stars.8 total votes.,FREE Shipping,NaN,Design: Sofa Bed; Standard,NaN,7940,"2,750.00 - $4,140.00"
8102,Vercelli Reclining Sofa,https://www.wayfair.com/furniture/pdp/omnia-le...,by Omnia Leather,NaN,2.0,Rated 5 out of 5 stars.2 total votes.,FREE Shipping,You will enjoy the comfort provided by the ove...,Overall: 82'' W x 40'' D,NaN,8102,"2,350.00 - $4,240.00"


In [11]:
price_range['new_price'] = (price_range['new_price'].str.split("\s-", n = 1, expand = True))[0]
price_range['new_price']

5898       479.99
6859     1,949.00
6879     2,569.99
6988     2,899.99
7027     2,239.99
7272     2,669.99
7363     2,250.00
7735     2,450.00
7940     2,750.00
8102     2,350.00
8235     4,099.99
8511     1,750.00
8594     2,200.00
8694     2,459.99
9219     1,869.99
9340     2,700.00
9343     1,769.99
9405     1,699.99
10369    2,250.00
10966    2,359.99
11216    2,399.99
11470    2,450.00
11501    2,499.99
11951    2,059.99
14813    2,000.00
14814    2,600.00
14815    2,500.00
14816    2,200.00
14819    2,499.99
16727      139.00
Name: new_price, dtype: object

In [12]:
df_new.loc[price_range.index] = price_range

In [13]:
df_new['new_price'] = df_new.new_price.str.replace(',', '').astype(float)

In [14]:
bins = [0,500,1000,1500,2000,2500,3000,3500,4000,4500,35000]
labels = ["<$500","$500-$1000","$1000-$1500","$1500-$2000","$2000-$2500","$2500-$3000","$3000-$3500","$3500-$4000","$4000-$4500",">$4500"]
df_new['price_bracket'] = pd.cut(df_new['new_price'], bins, labels=labels)

In [15]:
features = ["product_data1","product_data2","product_descrip","manufacturer"]

In [16]:
for feature in features:
    df_new[feature] = df_new[feature].fillna('')

In [17]:
def combine_features(row):
    try:
        return row['product_data1'] +" "+row['product_data2']+" "+row["product_descrip"]+" "+row["price_bracket"]+" "+row["manufacturer"]
    except:
        print("Error:", row)

In [18]:
df_new["combined_features"] = df_new.apply(combine_features,axis=1)

In [19]:
df_new.head()

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price,price_bracket,combined_features
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,,0,38.02,<$500,Switch Type: On/Off switch Short on space? Ma...
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1,73.99,<$500,Switch Type: Foot Switch Maximum Wattage (per ...
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2,104.99,<$500,Switch Type: Rotary socket Maximum Wattage (pe...
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3,143.99,<$500,Switch Type: 4-Way Maximum Wattage (per Bulb):...
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4,62.99,<$500,Switch Type: On/Off switch Maximum Wattage (pe...


In [20]:
stop_words = stopwords.words('english')

In [21]:
df_new['combined_features'] = df_new['combined_features'].str.lower().str.split()

In [22]:
df_new.head()

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price,price_bracket,combined_features
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,,0,38.02,<$500,"[switch, type:, on/off, switch, short, on, spa..."
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1,73.99,<$500,"[switch, type:, foot, switch, maximum, wattage..."
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2,104.99,<$500,"[switch, type:, rotary, socket, maximum, watta..."
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3,143.99,<$500,"[switch, type:, 4-way, maximum, wattage, (per,..."
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4,62.99,<$500,"[switch, type:, on/off, switch, maximum, watta..."


In [23]:
df_new["features"]=df_new["combined_features"].apply(lambda x: [word for word in x if word not in stop_words])

In [24]:
df_new.head()

,product_name,product_name_link,manufacturer,original_price,review_count,review_data,shipping_time,product_descrip,product_data1,product_data2,index,new_price,price_bracket,combined_features,features
0,"Ariana 63"" Column Floor Lamp",https://www.wayfair.com/lighting/pdp/zipcode-d...,by Zipcode Design,69.99,3878.0,Rated 4.5 out of 5 stars.3878 total votes.,NaN,Short on space? Make the most of your square f...,Switch Type: On/Off switch,,0,38.02,<$500,"[switch, type:, on/off, switch, short, on, spa...","[switch, type:, on/off, switch, short, space?,..."
1,"Dale 63.75"" Arched/Arc Floor Lamp",https://www.wayfair.com/v/wayup/sponsored_skus...,by Langley Street,99.99,1382.0,Rated 4.5 out of 5 stars.1382 total votes.,FREE 2-Day Shipping,"With a slender silhouette, this floor lamp bri...",Switch Type: Foot Switch,Maximum Wattage (per Bulb): 100 Watt,1,73.99,<$500,"[switch, type:, foot, switch, maximum, wattage...","[switch, type:, foot, switch, maximum, wattage..."
2,"Carlisle Douthit 70"" Swing Arm Floor Lamp",https://www.wayfair.com/lighting/pdp/williston...,by Williston Forge,399.99,1084.0,Rated 4.5 out of 5 stars.1084 total votes.,FREE 2-Day Shipping,"Part decor and part luminary, we love floor la...",Switch Type: Rotary socket,Maximum Wattage (per Bulb): 100 Watt,2,104.99,<$500,"[switch, type:, rotary, socket, maximum, watta...","[switch, type:, rotary, socket, maximum, watta..."
3,"Morrill 82"" Tree Floor Lamp",https://www.wayfair.com/lighting/pdp/brayden-s...,by Brayden Studio,230.00,3476.0,Rated 4.5 out of 5 stars.3476 total votes.,FREE 2-Day Shipping,When your ensemble is in need of a little illu...,Switch Type: 4-Way,Maximum Wattage (per Bulb): 26 Watt,3,143.99,<$500,"[switch, type:, 4-way, maximum, wattage, (per,...","[switch, type:, 4-way, maximum, wattage, (per,..."
4,"Shipton Crystal and Metal 61"" Floor Lamp",https://www.wayfair.com/lighting/pdp/willa-arl...,by Willa Arlo Interiors,170.04,2099.0,Rated 4.5 out of 5 stars.2099 total votes.,FREE 2-Day Shipping,Equally alluring sitting sofa-side for a littl...,Switch Type: On/Off switch,Maximum Wattage (per Bulb): 60 Watt,4,62.99,<$500,"[switch, type:, on/off, switch, maximum, watta...","[switch, type:, on/off, switch, maximum, watta..."


In [25]:
df_new["features"]=df_new["features"].apply(lambda x: " ".join(x))

In [26]:
cv = CountVectorizer()

In [27]:
count_matrix = cv.fit_transform(df_new["features"])


In [28]:
count_matrix

<17209x13136 sparse matrix of type '<class 'numpy.int64'>'
	with 607000 stored elements in Compressed Sparse Row format>

In [29]:
cosine_sim = cosine_similarity(count_matrix)

In [30]:
product_user_likes = 'Bjorn Chesterfield Settee'

In [31]:
product_index = get_index_from_title(product_user_likes)

similar_products =  list(enumerate(cosine_sim[product_index]))

In [32]:
sorted_similar_products = sorted(similar_products,key=lambda x:x[1],reverse=True)

In [33]:
i=0
for element in sorted_similar_products:
        print (get_title_from_index(element[0]),get_home(element[0]))
        i=i+1
        if i>50:
            break

Bjorn Chesterfield Settee https://www.wayfair.com/furniture/pdp/andover-mills-bjorn-chesterfield-settee-w000619313.html
Kaitlyn Settee https://www.wayfair.com/furniture/pdp/andover-mills-kaitlyn-settee-w000539497.html
Thame Swoop Arm Settee Sofa & Chaise https://www.wayfair.com/furniture/pdp/brayden-studio-thame-swoop-arm-settee-sofa-chaise-bydt1178.html
Forest River Tufted Loveseat https://www.wayfair.com/furniture/pdp/forest-river-tufted-loveseat-lfmf3375.html
Omar Loveseat https://www.wayfair.com/furniture/pdp/charlton-home-omar-loveseat-cnta1632.html
Jonas Loveseat https://www.wayfair.com/furniture/pdp/andover-mills-jonas-loveseat-ando3610.html
Elstone Chesterfield Sofa https://www.wayfair.com/furniture/pdp/willa-arlo-interiors-elstone-chesterfield-sofa-wrlo7049.html
Shullsburg Loveseat https://www.wayfair.com/furniture/pdp/gracie-oaks-shullsburg-loveseat-w000498808.html
Marin Loveseat https://www.wayfair.com/furniture/pdp/mistana-marin-loveseat-w000112042.html
Nyx Sofa https://www